INSTALLING DEPENDENCIES

In [ ]:
pip install numpy
pip install scipy
pip install pyrtlsdr
pip install tensorflow
pip install scikit-learn
pip install matplotlib.pyplot

RADIO SIGNALS COLLECTION AND SAMPLING

In [ ]:
from rtlsdr import RtlSdr

# Initialize the RTL-SDR
sdr = RtlSdr()

# Configure the RTL-SDR
sdr.sample_rate = 2.048e6  # 2.048 MHz
sdr.center_freq = 2.4e9  # 2.4 GHz (Wi-Fi example frequency)
sdr.gain = 'auto'

# Capture samples
samples = sdr.read_samples(256*1024)  # Capture 256k samples

# Save samples to a file
np.save('rf_samples.npy', samples)

# Close the RTL-SDR
sdr.close()

SCALING WITH HYBRID(SAMPLES+SYNTHETIC)DATA TO CAPTURE NOISE

In [ ]:
import numpy as np
from scipy.fftpack import fft
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

def generate_synthetic_rf_data(num_samples=1000, num_classes=10):
    data = []
    labels = []
    for i in range(num_classes):
        for _ in range(num_samples):
            # Generate a synthetic signal with noise
            signal = np.sin(2 * np.pi * np.random.rand() * np.arange(256)) + np.random.normal(0, 0.5, 256)
            data.append(signal)
            labels.append(i)
    return np.array(data), np.array(labels)

# Generate synthetic RF data
X, y = generate_synthetic_rf_data()

# Visualize a sample signal
plt.plot(X[0])
plt.title(f"Class {y[0]} Signal")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()

DATA PREPROCESSING USING FFT

In [ ]:
def preprocess_rf_samples(data):
    # Convert samples to the frequency domain using FFT
    transformed_samples = np.abs(fft(data))

    # Normalize the data
    scaler = StandardScaler()
    scaled_samples = scaler.fit_transform(transformed_samples.reshape(-1, 1))

    return scaled_samples.reshape(data.shape)

# Preprocess the synthetic RF data
preprocessed_data = preprocess_rf_samples(X)

CNN MODEL DEFINITION AND TRAINING TO DETECT RADIO FREQUENCY SIGNALS

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

input_shape = (256, 256, 1)
X_train = preprocessed_data.reshape(-1, 256, 256, 1)
y_train = y

model = build_cnn_model(input_shape)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)
model.save("rf_signal_detector.h5")


MODEL LOADING AND FINAL PREDICTION

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("rf_signal_detector.h5")

X_new, y_new = generate_synthetic_rf_data(num_samples=10)
preprocessed_new_data = preprocess_rf_samples(X_new)

predictions = model.predict(preprocessed_new_data.reshape(-1, 256, 256, 1))
predicted_classes = np.argmax(predictions, axis=1)

for i, pred_class in enumerate(predicted_classes):
    print(f"Sample {i}: True Class = {y_new[i]}, Predicted Class = {pred_class}")